# Automate Medium Story Creation with Python
In this tutorial, we will learn how to automate the process of creating a Medium story using the Medium API. This will allow you to programmatically publish stories on your Medium account.

---

Here is an overview of the steps we will take to create a story using the Medium API in Python:
1. [Prerequisites](#1-prerequisites)
2. [Function to do API call](#2-function-to-do-api-call)
3. [Function to create a story](#3-function-to-create-a-story)
4. [Create a story](#4-create-a-story)
5. [Optional: Create a Story from a Jupyter Notebook](#5-optional-create-a-story-from-a-jupyter-notebook)

## 1. Prerequisites

To use the Medium API, you need to follow these steps to generate the token:
1. Go to "Settings" Page by clicking on your profile picture at the top right corner of the medium home page.
2. Under "Security and apps" tab, click on "Integration tokens" at the bottom of the page.
3. Generate a new token by clicking on "Get token" button.
4. Create a new file named `.env` in the root directory of this project and paste the token in it as follows:
    ```
    MEDIUM_TOKEN="<Your_Token>"
    ```

### 1.1 Load Environment Variables 

Now, let's load the environment variables from the `.env` file. This step is essential for authenticating our API calls.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

## 2. Function to do API call

To interact with the Medium API, we need to create a function that can send HTTP requests to the API endpoints. We'll use the `requests` library to achieve this.

In [ ]:
import requests
from typing import Optional, Dict, Any, List


def send_request(
    url: str,
    method: str = "get",
    body: Optional[Dict[str, Any]] = None,
    headers: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    """
    Send a request to the Medium API

    :param url: The URL to send the request to
    :param method: The HTTP method to use
    :param body: The body of the request
    :param headers: The headers of the request
    :return: The response of the request
    """
    response = requests.request(method, url, json=body, headers=headers)
    return response.json()

## 3. Function to Create a Story

We'll create a function that allows us to create a new Medium story using the authenticated user's account. Before calling this function, ensure you have set the `MEDIUM_TOKEN` environment variable from the prerequisites step.

In [ ]:
user_id = None

In [ ]:
def create_post(
    title: str,
    content: str,
    content_format: str = "markdown",
    tags: Optional[List[str]] = None,
    canonical_url: Optional[str] = None,
    publish_status: str = "draft",
) -> Dict[str, Any]:
    """
    Create a post on Medium.

    :param title: The title of the post, used for SEO, doesn't appear in the actual post.
    Should be less than 100 characters.
    :param content: The content of the post.
    :param content_format: The format of the content, either `html` or `markdown`, defaults to `markdown`.
    :param tags: The tags of the post, used to classify the post. Only the first three will be used.
    Should be less than 25 characters each. Defaults to `None`.
    :param canonical_url: The original url of this content, if it was originally published elsewhere.
    Defaults to `None`.
    :param publish_status: The publish status of the post, either `public`, `draft`, or `unlisted`,
    defaults to `draft`.
    :return: The response of the request.
    """
    global user_id

    # Get the user_id
    if user_id is None:
        my_details_url = "https://api.medium.com/v1/me"
        headers = {"Authorization": f"Bearer {os.environ.get('MEDIUM_TOKEN')}"}

        response = send_request(my_details_url, headers=headers)
        user_id = response.get("data").get("id")

    # Ready to create a post
    url = f"https://api.medium.com/v1/users/{user_id}/posts"

    # set headers
    headers = {"Authorization": f"Bearer {os.environ.get('MEDIUM_TOKEN')}"}

    # set body
    body = {
        "title": title,
        "contentFormat": content_format,
        "content": content,
        "tags": tags,
        "canonicalUrl": canonical_url,
        "publishStatus": publish_status,
    }

    # send request with body and authentication bearer token
    response = send_request(url, method="post", body=body, headers=headers)
    return response

## 4. Create a Story

Now, let's create a new Medium story using the `create_post` function. Modify the variables as per your need.

In [ ]:
title = "1st Test Post"
content = "# Heading! \n## Sub-Heading! \n Welcome to my first post!"
content_format = "markdown"
tags = ["test", "python", "medium"]
canonical_url = "https://medium.com/@xyz"
publish_status = "draft"

response = create_post(
    title, content, content_format, tags, canonical_url, publish_status
)
response

## 5. Optional: Create a Story from a Jupyter Notebook

### 5.1 Function to Convert Notebook to Markdown

This function allows you to convert a Jupyter Notebook to Markdown.

In [ ]:
import requests
import nbformat
from nbconvert import MarkdownExporter


def notebook_url_to_markdown(url: str) -> str:
    """
    Convert a Jupyter Notebook to Markdown.

    :param url: The URL of the Jupyter Notebook.
    :return: The Markdown content of the Jupyter Notebook.
    """
    # Fetch the Jupyter Notebook from the URL
    response = requests.get(url)
    notebook_content = response.text

    # Parse the notebook using nbformat
    notebook = nbformat.reads(notebook_content, as_version=nbformat.NO_CONVERT)
    metadata = notebook.get("metadata", {})

    # Convert the notebook to Markdown
    md_exporter = MarkdownExporter()
    markdown, _ = md_exporter.from_notebook_node(notebook, resources=metadata)

    return markdown

> Note: Please copy the **raw url** of the notebook from GitHub.

Modify the variables as per your need.

In [ ]:
# url from github must be raw url
notebook_url = "https://raw.githubusercontent.com/iamrk04/TaskAutomationKit/main/notebooks/create-medium-story-automated.ipynb"
markdown_content = notebook_url_to_markdown(notebook_url)

### 5.2 Create a Story from Markdown

Modify the variables as per your need.

In [ ]:
title = "Automate Medium Story Creation with Python"
content = markdown_content
content_format = "markdown"
tags = ["python", "medium", "automation"]
canonical_url = "https://github.com/iamrk04/TaskAutomationKit/blob/main/notebooks/create-medium-story-automated.ipynb"
publish_status = "draft"

response = create_post(
    title, content, content_format, tags, canonical_url, publish_status
)
response